# Goal
Predict whether the accepted loan would be properly paid back or not

log:
drop columns and load all data
move on to feature engineering


Make it better:
Can we use forward and backward or stepwise method to select the important parameter?

Should we treat ordinal value as catagory values or numeric values?


## import data

In [1]:
# load library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Data Cleansing

In [ ]:
# import data
reader = pd.read_csv('accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv', chunksize=10000)
mid=[]
for chunk in reader:
    chunk=chunk[chunk['loan_status'].apply( lambda x: x not in ['Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off'])]
    # drop columns with more than 50% missing values
    chunk.rename(columns={'verification_status_joint':'verified_status_joint'}, inplace=True)

    chunk = chunk.drop(columns= ['verified_status_joint','sec_app_mths_since_last_major_derog', 'sec_app_revol_util',
       'revol_bal_joint', 'sec_app_inq_last_6mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_chargeoff_within_12_mths', 'sec_app_num_rev_accts',
       'sec_app_open_acc', 'sec_app_mort_acc', 'sec_app_fico_range_high',
       'sec_app_fico_range_low', 'dti_joint', 'annual_inc_joint',
       'mths_since_last_record', 'mths_since_recent_bc_dlq',
       'mths_since_last_major_derog', 'mths_since_recent_revol_delinq',
       'mths_since_last_delinq'])

    #  drop columns with more than 20% of missing values
    chunk = chunk.drop(columns=['open_acc_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m'])

    # drop column without explaination
    chunk = chunk.drop(columns=['debt_settlement_flag', 'debt_settlement_flag_date', 'deferral_term',\
         'disbursement_method', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 'hardship_flag',\
            'hardship_last_payment_amount', 'hardship_length', 'hardship_loan_status', 'hardship_payoff_balance_amount', \
            'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type', 'open_act_il', \
            'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 'sec_app_open_act_il',\
            'settlement_amount', 'settlement_date', 'settlement_percentage', 'settlement_status', 'settlement_term'])
    # drop column with high correlation
    chunk = chunk.drop(columns=['out_prncp_inv','funded_amnt','funded_amnt_inv','tot_hi_cred_lim','total_il_high_credit_limit'])
    # columns that are not related to prediction
    chunk = chunk.drop(columns=['url','desc','member_id','id','emp_title','sub_grade','zip_code','policy_code'])

    # drop column with date
    chunk = chunk.drop(columns= ['issue_d','earliest_cr_line','last_pymnt_d','next_pymnt_d','last_credit_pull_d','sec_app_earliest_cr_line'])

    mid.append(chunk)

accepted = pd.concat(mid)

In [ ]:
accepted.shape

In [ ]:
# check the imported data
accepted.head()

In [ ]:
# description from dataset:
data_descriptions = pd.read_excel('LCDataDictionary.xlsx', sheet_name=[0,1,2])

## Data Wrangling


In the following, we will try to analysis the column information and remvovee non-related columns. To do this, our first step is to understand the meaning of each column. The data set is acompanied with a well-documented data description. We will use it as a guideline.

### Remove columns that are not well defined

In [ ]:
#data_descriptions[1] = data_descriptions[1].iloc[:,0:2]

In [ ]:
# concat the descriptions to one sheet
data_descriptions[0] = data_descriptions[0][['LoanStatNew', 'Description']]
data_descriptions[1].columns =['LoanStatNew', 'Description']
data_descriptions = pd.concat([data_descriptions[0], data_descriptions[1]], ignore_index=True)

In [ ]:
# clearly there is an extra space need to be dealt with.
data_descriptions.LoanStatNew = data_descriptions.LoanStatNew.str.strip()
# verified status joint is called verification status joint in the dataset
accepted.rename(columns={'verification_status_joint':'verified_status_joint'}, inplace=True)

In [ ]:
# remove na
data_descriptions = data_descriptions[[pd.notna(i) for i in data_descriptions.LoanStatNew]]

In [ ]:
# check which column does not have an description
print('column name in data without description')
print(sorted(set(accepted.columns)-set(data_descriptions.LoanStatNew))) # col in data without description
print('*'*100)
print(sorted(set(data_descriptions.LoanStatNew)-set(accepted.columns)  )) # columns with description not in data
# The columns which does not contain a proper description will be removed. 

In [ ]:
# drop columns that do not have a proper explaination
accepted.drop(columns=list(set(accepted.columns)-set(data_descriptions.LoanStatNew)), inplace=True)
accepted.head()

### general examine data type

In [ ]:
# earliest_cr_line, last_pymnt_d, next_pymnt_d, last_credit_pull_d suppose to be datetime month_label-Year
#for col in ['earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d','issue_d','sec_app_earliest_cr_line']:
#    accepted[col] = pd.to_datetime(accepted[col], format='%b-%Y')

In [ ]:
# convert id into object
#accepted[['id','policy_code']]=accepted[['id','policy_code']].astype('object')

In [ ]:
# check existing dtypes
accepted.dtypes.unique()

In [ ]:
# int
accepted.select_dtypes('int64').columns # id should be cat value
# <M8[ns]
datetime_dict={col: [ accepted[col].unique()] for col in accepted.select_dtypes('<M8[ns]').columns}
# float
float_dict={col: [ accepted[col].unique()] for col in accepted.select_dtypes('float64').columns}
# 0
object_dict={col: [accepted[col].unique()] for col in accepted.select_dtypes('O').columns}


### unique datetime
would not necessary for prediction

potential: time interval 

In [ ]:
# unique datetime, generate date_col
pd.DataFrame.from_dict(datetime_dict, orient='index').rename(columns={0:'unique_values'})
date_col = ['issue_d','earliest_cr_line','last_pymnt_d','next_pymnt_d','last_credit_pull_d','sec_app_earliest_cr_line']

### unique object values
- identify columns that are not necessary: 
id, url, zip_code, grade/subgrade, title?, emp_title?


- test whether data type is proper or not
- define values for prediction
- clean nan values
- chisq independent test features

In [ ]:
# unique object values
object_dict_unique = pd.DataFrame.from_dict(object_dict, orient='index').rename(columns={0:'unique_values'})
object_dict_unique['n']=object_dict_unique['unique_values'].apply(len)
object_dict_unique.sort_values("n")
# not related columns: id, url, desc

#### clean title (deal with unstructured data)

In [ ]:
# check the value that contains loan
accepted['title'].str.lower().str.replace(' loan','').str.contains('loan').value_counts()

In [ ]:
# create list of stopwords and remove stopwords
stop_words1=['just','a','the','my','need','help','to', 'one', 'me', 'for','smart', 'jc', 'low', 'looking', 'lower', 'mike\'s', 'mike',\
    'michelle\'s', 'michelle', 'many', 'needed', 'mission', 'dad\'s', 'seeking', 'high', 'it', 'new', 'nice', 'and','quick','next','level', 'more',\
        'large','small','lendingclub','better', 'me', 'you','beautiful','easy', 'finally', 'rescue', 'get','first','last','second', 'up','lower',\
            'combine','little', 'project','please', 'thank', 'thanks','ny','of','is','are','i','on','&','this','in','me,','be','with','from','-',\
                'big','short','end','our', 'needs', 'bye','two','over','will','at','some','do','clear','combine','no','or']
accepted['title'] = accepted['title'].apply( lambda x: ' '.join([s for s in x.split() if (s not in stop_words1)]) if pd.notna(x) else x )

In [ ]:
#### value recode
stop_words2=['lending club','no more','-','\s','\d','$','best','better','big','bye','final','buy','finish','going','good','great','hard','high',\
    'happy','honest','less','than','responsible','rich','right','short','long','smart','smile','term','want','unexpected','the','project','new','mine',
    'profit','first','second','expense']
accepted['title'] = accepted['title'].str.lower().str.replace(' loan','').str.replace('loan ','').str.strip('!,\'-.?&#$\/\ \\0123456789%:+=\"_)(')
accepted['title'] = accepted['title'].str.replace('c.c.','credit card')
accepted['title'] = accepted['title'].str.replace('lc','lending club')
accepted['title'] = accepted['title'].str.replace('cc','credit card')

for word in stop_words2:
    accepted['title'] = accepted['title'].str.replace(word,'')

In [ ]:
# examine rows that contain certain word
#### grade and subgrade, if grade is empty, subgrade is empty
#accepted[['grade', 'sub_grade']].isna().sum(axis=1).unique()
#### moving is basically physically moving
accepted['title'][accepted['title'].apply(lambda x: 'moving' in x if pd.notna(x) else False)]
#### mustang is car
accepted['title'][accepted['title'].apply(lambda x: 'mustang' in x if pd.notna(x) else False)]
#### banks means people dont like the idea of bank
accepted['title'][accepted['title'].apply(lambda x: 'banks' in x if pd.notna(x) else False)]
#### mediacal is medial expanse
accepted['title'][accepted['title'].apply(lambda x: 'med' in x if pd.notna(x) else False)]
#### mba is school
accepted['title'][accepted['title'].apply(lambda x: 'mba' in x if pd.notna(x) else False)]

In [ ]:
accepted['title'][accepted['title'].apply(lambda x: 'wash' in x if pd.notna(x) else False)].value_counts().head(40)

In [ ]:
# create 10 categories for recode based on the rule
reword={'debt':['deb','debt','dedt','dbt','dept','bill','debit','wells','chase','citi','visa','bankamerica','bankofamerica','barclay','amex','american','boa','bofa','credit card',\
        'creditcard','credit','card','pay','pay-off','payoff','pay off','off','payback','paid','discover'],\
    'consolidation':['consolidation','con','cos','capitolslate','consalidadtion','conso','reconciliation','consoildation','consolidation','consolodation',\
        'consoldate','consolitation','consolidate','consol','cosolidation','onsolidation','recon','consol','refinance','refi','re-fi'],\
    'medical':['med','hospital','dental','health','surgery','dentist','rehab','headache','doctor'],\
    'wedding':['diamondring','wed','engage','honey','wedding','engagement','marr','ring'],\
    'mbuy':['mus','major','equip','daniel','defense','appliance','computer','laptop','camera','purchase','perchase','porchase',\
        'golf','boat','purchase','gun','software','seadoo','ship'],\
    'vehicle':['motor','dodge','harley','kawa','chevy','bmw','ford','toyota','wheels','honda','scooter','vehic','truck','mustang','subaru','suzuki',
        'mazda','auto','mercedes','auto','fuel','car','transmission','jeep','bike','trailer','subaru','nissan','engine','volvo','truck'],\
    'emergency':['emer','emr','emergency'],\
    'moving':['crossc','relo','moving','move'],\
    'law':['legal','attorney','law'],\
    'school':['edu','exam','training','classes','school','mba','mster\'s','student','graduate','phd','education','course','tuition','book','college',
        'teacher','program'], 
    'business':['farm','invent','business','buiness','buis','bus','bakery','shop','studio','web','buisness','company','busines','start-up',
        'startup','start up','inves'],\
    'home':['hous','heat','condo','chimney','apart','build','barn','basement','bassment','bath','boil','borrow',\
        'driveway','sewer','solar','property','cabin','yard','office','lawn','basement','renovat','home','mortgage','tub','pool','roof','rent',\
        'garage','bathroom','bedroom','kitchen','outdoor','suite','room','floor','ceil','garden','house','window','deck','fence','remodel','a/c','furniture','bed',
        'furnace','landscape','shelter','remo','tree','wash','lighttunnel'],\
    'personal':['presonal','pesonel','priv','peronal','personnel','personal','person','personal','vacation','money','cash','trip','pers','pes'],\
    'family':['child','adop','brother','sister','baby','mom','father','mother','grand','dad','daughter','kid','fam','funeral','myson'],\
    'other':['making','catch','eas','chan','fix','hop','impr','clear','com','never','add','bad','break','bright','help',\
        'bridge','sum','bless','blue','back','balan','insurance','together','all','goal','god','no','temp','self','free',\
        'live','lend','start','breath','day','clean','dream','love','peace','jan','feb','march','april','may','june','july',\
        'august','september','october','november','december','financ','redu','stres','soul','luck','month','opera','reduce',\
        'spring','time','out','plan','clos','capi','life','begin','mistake','sav','relief','air','tax','apr','interest','banks','green','vaction','simp']
        }



In [ ]:
# recode all the values to 10 category.
for k,v in reword.items():
    for item in v:
        accepted['title']=accepted['title'].apply(lambda x: (k if item in x else x) if isinstance(x,str) else x)


In [ ]:
# replace certain values to other
accepted['title'] = accepted['title'].replace({'or':'other','loan':'other','':'other','myloan':'other','future':'other','s':'other','k':'other',
'my':'other'})

In [ ]:
# unique value of each category
test = accepted[['title']].value_counts()
plt.plot(range(len(test)), test)

In [ ]:
# rename value to other if they are not in the top 10 categories
top10_index = [col[0] for col in accepted[['title']].value_counts().iloc[:10].index]
accepted['title'] = accepted['title'].apply(lambda x: (x if x in top10_index else 'other') if isinstance(x,str) else x)

In [ ]:
#### check the cleaned categories.
accepted[['title']].value_counts().head(20)
#[test.append(i[0].split()) if isinstance(i[0].split(),str) else test.extend(i[0].split()) for i in accepted[['title']].value_counts().keys()]
#from collections import Counter
#sorted(Counter(test).items(), key=lambda x: x[1], reverse=True)
#sorted(Counter(test).items(), key=lambda x: x[0])[6000:]

In [ ]:
#### emp title 436961 unique
#len(accepted['emp_title'].str.lower().value_counts())

In [ ]:
# examine unique value in each object col and generate col name not reelated to the prediction.
not_related_cols = ['id','emp_title','sub_grade','zip_code','policy_code']
object_dict_unique

#### define loan status value

**Column Explaination**

revolving loan: A revolving loan facility is a form of credit issued by a financial institution that provides the borrower with the ability to draw down or withdraw, repay, and withdraw again. A revolving loan is considered a flexible financing tool due to its repayment and re-borrowing accommodations.


**Loan status explaination**

Current: Loan is up to date on all outstanding payments. / any loan that is fully paid to date according to a contract.

Fully paid: Loan has been fully repaid, either at the expiration of the 3- or 5-year year term or as a result of a prepayment.
 
 
In Grace Period: Loan is past due but within the 15-day grace period. 
 
Late (16-30): Loan has not been current for 16 to 30 days. Learn more about the tools LendingClub has to deal with delinquent borrowers.
 
Late (31-120): Loan has not been current for 31 to 120 days. Learn more about the tools LendingClub has to deal with delinquent borrowers.
 
Default: Loan has not been current for an extended period of time. 

Charged Off: Loan for which there is no longer a reasonable expectation of further payments. Upon Charge Off, the remaining principal balance of the Note is deducted from the account balance.

Default and charged-off:
About 4-6 months after you miss your first payment, your loan will default and then charge-off. 

**From the defination above, charged off and default would be treated as default**


In [ ]:
#check unique value counts in the column we want to predict
accepted['loan_status'].value_counts()
# From the table bellow, does not meet the credit policy would be removed

In [ ]:
# remvove rows that loan_status contains nan
accepted.dropna(subset=['loan_status'],inplace=True)

In [ ]:
# if charged off or default then treat as default, rest are not default
accepted['Default']=accepted['loan_status'].apply(lambda x: 1 if x in ['Default', 'Charged Off'] else 0).astype('object')

#### fill missing values

In [ ]:
# check the percentage of missing values
accepted[object_dict_unique.index].isnull().mean()*100
# drop verified_status_joint has more than 94 percent of missing values

In [ ]:
# fill missing cat value with mode
#accepted.drop(columns='loan_status', inplace=True)
accepted['emp_length']=accepted['emp_length'].fillna(accepted['emp_length'].mode()[0])
accepted['title']=accepted['title'].fillna(accepted['title'].mode()[0])

### unique numeric values

In [ ]:
# unique numeric values
float_dict_unique=pd.DataFrame.from_dict(float_dict, orient='index').rename(columns={0:'unique_values'})
float_dict_unique['n']=float_dict_unique['unique_values'].apply(len)

In [ ]:
# find column with single value
float_dict_unique[float_dict_unique['n']==1].index
# single value: 'member_id'

In [ ]:
# find column with less than 10% of unique values. (1000 for 10000)
unique_index_1000=float_dict_unique[float_dict_unique['n']>1000].index

In [ ]:
# display all columns
single_value_index=[]

numeric_cols=float_dict_unique.drop(index=single_value_index)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(numeric_cols.sort_values('n'))

# policy_code is cat value

#### check missing values

In [ ]:
# percentage of 0s
float_zeros = pd.DataFrame((accepted[float_dict_unique.index]==0).mean())
float_zeros.columns=['percentage of 0s']

In [ ]:
# remove all missing values
accepted.isnull().any(axis=1).sum()/2257952 # 0.9998609359277788 not possible to remove all rows.

In [ ]:
#  remove columns with missing  values more than 50%
float_dict_missing = accepted[float_dict_unique.index].isnull().mean()*100
float_dict_missing = float_dict_unique.join(float_dict_missing.rename('missing percentage'))

In [ ]:
# histogram of misssing
float_dict_missing['missing percentage'].sort_values().plot(kind='bar')
plt.ylabel('percentage of missing values')
plt.xlabel('column names')

In [ ]:
# rows with missing value more than 20%s
float_dict_missing[float_dict_missing['missing percentage']>20].index

In [ ]:
#  check the number of rows with full set of data
accepted[float_dict_missing.index].isnull().any(axis=1).sum()

In [ ]:
'''
    def df_unique_missing_0s(df):
    numeric_cols=float_dict_unique.drop(index=single_value_index)
    # percentage of 0s
    float_zeros = pd.DataFrame((df[float_dict_unique.index]==0).mean())
    float_zeros.columns=['percentage of 0s']
    # missings
    float_dict_missing = df[float_dict_unique.index].isnull().mean()*100
    float_dict_missing = float_dict_unique.join(float_dict_missing.rename('missing percentage'))
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(numeric_cols.join(float_zeros).join(float_dict_missing['missing percentage']).\
        sort_values(['missing percentage','percentage of 0s'], ascending=False))
'''

In [ ]:
#accepted_trimed=accepted.drop(columns='mths_since_recent_inq').dropna()
# could not use trimed version because it significantly remove default data

In [ ]:
# summary of unique value, percentage of 0s in the list, and persentage of missing values.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(numeric_cols.join(float_zeros).join(float_dict_missing['missing percentage']).\
        sort_values(['missing percentage','percentage of 0s'], ascending=False))

#### check the correlation value

In [ ]:
accepted['Default'].value_counts()

In [ ]:
# check the correlation of the column that contains numeric values
accepted_numeric_corr = accepted[unique_index_1000].drop(columns=['recoveries','collection_recovery_fee']).corr()

plt.figure(figsize=(15,10))
sns.heatmap(accepted_numeric_corr,vmin=-1, vmax=1)

In [ ]:
# generate colname and correlations
accepted_numeric_corr_melt = accepted_numeric_corr.reset_index(drop=False).melt('index', var_name='var2', value_name='corr')
accepted_numeric_corr_melt = accepted_numeric_corr_melt.rename(columns={'index':'var1'})
highly_corr_cols = accepted_numeric_corr_melt[(abs(accepted_numeric_corr_melt['corr'])>0.5) & (accepted_numeric_corr_melt['var1']!=accepted_numeric_corr_melt['var2'])]
highly_corr_cols['combine'] = (highly_corr_cols['var1']+highly_corr_cols['var2']).apply(lambda x: ''.join(sorted(x)))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    highly_corr_cols=highly_corr_cols.sort_values('corr', ascending=False).drop_duplicates(subset='combine')
    display(highly_corr_cols)

In [ ]:
# plot correlation with corr>0.5
for v1,v2 in highly_corr_cols[['var1','var2']].values:
    print(v1,v2)
    plt.figure()
    #plt.scatter(accepted[v1], accepted[[v1,v2]].groupby(v1).mean().reset_index()[v2])
    accepted[[v1,v2]].groupby(v1).mean().reset_index().plot(kind='scatter', x=v1,y=v2)
    plt.show()
# drop on in:
# out_prncp/out_prncp_inv; funded_amnt/loan_amnt; loan_amnt/funded_amnt_inv; tot_cur_bal/tot_hi_cred_lim;
# total_il_high_credit_limit/total_bal_ex_mort; 


In [ ]:
# out_prncp/out_prncp_inv; tot_cur_bal/tot_hi_cred_lim;
# total_il_high_credit_limit/total_bal_ex_mort; 
high_corr_cols = ['out_prncp_inv','funded_amnt','funded_amnt_inv','tot_hi_cred_lim','total_il_high_credit_limit']

**delinquent**: Delinquency means that you are behind on payments. Once you are delinquent for a certain period of time (usually nine months for federal loans), your lender will declare the loan to be in default. The entire loan balance will become due at that time.

**trades**: Trade finance represents the financial instruments and products that are used by companies to facilitate international trade and commerce.

#### fill missing

In [ ]:
float_dict_missing[float_dict_missing['missing percentage']>0].sort_values('n', ascending=False)
# below 20 use mode, above 20 use median
for col in float_dict_missing.index:
    if float_dict_missing.loc[col,'n']>20:
        accepted[col].fillna(accepted[col].median(),inplace=True)
    else:
        accepted[col].fillna(accepted[col].mode()[0], inplace=True)

In [ ]:
# check if there is any missing value in the df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.DataFrame(accepted.isnull().mean()).sort_values(0))

## EDA

In [ ]:
# import cleaned dataset
accepted_cleaned = pd.read_csv('./cleaned dataset.csv')
accepted_cleaned.drop(columns=['Unnamed: 0'],inplace=True)
accepted_cleaned.head()

In [ ]:
np.unique(accepted_cleaned['title'])

In [ ]:
accepted_cleaned.info()

In [ ]:
# frequency of the of each type
for col in accepted_cleaned.select_dtypes('object').columns:
    sns.countplot(accepted_cleaned[col])
    plt.xticks(rotation = 90)
    plt.show()

In [ ]:
for col in accepted_cleaned.select_dtypes('O').columns:
    try:
        current_table = pd.pivot_table(accepted_cleaned, values='loan_amnt', index=col, columns='loan_status', aggfunc='count')
        sns.heatmap(current_table)
        plt.show()
    except:
        print(col)

In [ ]:
# boxplot of object
for col in accepted_cleaned.select_dtypes('float').columns:
    sns.boxplot(y=col, x='loan_status',  data=accepted_cleaned)
    plt.show()

## Feature Engineering

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
import xgboost as xgb

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV

In [ ]:
accepted_cleaned['term'] = accepted_cleaned['term'].str.strip()

In [ ]:
# split and check
X_train, X_test, y_train, y_test= train_test_split(accepted_cleaned.drop(columns='loan_status'), accepted_cleaned['loan_status'], test_size=0.75, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# Smote for imbalanced data
#from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
ColumnTransform = ColumnTransformer(transformers=[
    ('numeric', StandardScaler(), X_train.select_dtypes(include='number').columns.tolist()),
    ('categorical', OneHotEncoder(drop='first'), X_train.select_dtypes(include='category').columns.tolist())
])
X_train = ColumnTransform.fit_transform(X_train)
X_test = ColumnTransform.transform(X_test)

In [ ]:
sm = RandomUnderSampler()
X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
X_train.shape, y_train.shape

### Feature importance

In [ ]:
from sklearn.ensemble import RandomForestClassifier
FI_RF = RandomForestClassifier()
FI_RF.fit(X_train, y_train)
FI_RF_importances = FI_RF.feature_importances_


In [ ]:
from sklearn.linear_model import LogisticRegression
FI_LR  = LogisticRegression()
FI_LR.fit(X_train, y_train)

FI_LR_importances = FI_LR.coef_[0]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
FI_tree = DecisionTreeClassifier()
FI_tree.fit(X_train, y_train)
FI_tree_importance = FI_tree.feature_importances_

In [ ]:
from xgboost import XGBClassifier
FI_XGboost = XGBClassifier()
FI_XGboost.fit(X_train, y_train)
FI_XGboost_importance = FI_XGboost.feature_importances_

In [ ]:
plt.subplot(2,2,1); plt.plot(FI_RF.feature_importances_, label='Random Forest')
plt.subplot(2,2,2); plt.plot(np.abs(FI_LR.coef_[0]), label = 'Logistic Regression')
plt.subplot(2,2,3); plt.plot(FI_tree.feature_importances_, label = 'Decision Tree')
plt.subplot(2,2,4); plt.plot(FI_XGboost.feature_importances_, label = 'XGboost')
plt.show()

In [ ]:
minmax = MinMaxScaler()
selected_features= np.abs( \
    minmax.fit_transform(FI_RF_importances.reshape(-1,1)) * \
    minmax.fit_transform(FI_LR_importances.reshape(-1,1)) * \
    minmax.fit_transform(FI_tree_importance.reshape(-1,1)) * \
    minmax.fit_transform(FI_XGboost_importance.reshape(-1,1)))

In [ ]:
#sum(selected_features>0.001)
selected_idx = np.squeeze(np.argsort(selected_features, axis=0))
columns = accepted_cleaned.columns[selected_idx[-10:]]
print(columns)
plt.plot(np.sort(selected_features[-10:], axis=0))

In [ ]:
# create selected cols
selected_cols = selected_idx[-10:] # selected_features[-10:]

In [ ]:
selected_cols

## Model Fitting and Pipeline build


### resplit data

In [ ]:
X_train = X_train[:,selected_cols]
X_test = X_test[:,selected_cols]

### Random Forest

In [ ]:
Steps = [('RF', RandomForestClassifier())]
params = {'RF__n_estimators': [10], 'RF__criterion':['gini'], 'RF__max_depth':[4] }

In [ ]:
RF_C = Pipeline(steps=Steps)

In [ ]:
GridSearchCV(RF_C, param_grid=params ,cv=5)

In [ ]:
RF_C.fit(X_train, y_train)


In [ ]:
y_pred = RF_C.predict(X_test)

In [ ]:
print(classification_report(y_pred=y_pred, y_true=y_test))

In [ ]:
X_train.shape, y_test.shape

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True)
plt.show()

In [ ]:
y_prob = RF_C.predict_proba(X_test) # The first index refers to the probability that the data belong to class 0, and the second refers to the probability that the data belong to class 1.
false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(y_test, y_prob[:,1])


In [ ]:
sns.scatterplot(false_positive_rate1, true_positive_rate1)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LG = LogisticRegression()

In [ ]:
LG.fit(X_train, y_train)
y_pred = LG.predict(X_test)
y_prob = LG.predict_proba(X_test)

In [ ]:
y_test.shape, y_prob[:,1].shape

In [ ]:
fpr, tpr, _ = roc_curve([int(i) for i in y_test],  y_prob[:,1])
auc = roc_auc_score([int(i) for i in y_test], y_prob[:,1])

In [ ]:
sns.scatterplot(fpr,tpr)

### NN

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
from keras.models import load_model

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
NN_y_train = to_categorical(y_train)

In [ ]:
X_train.shape, NN_y_train.shape

In [ ]:
#Dense layers only
model=Sequential()
model.add(Dense(12,input_dim=63,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(2,activation='softmax')) # output the results into two categories and l
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc']) #optimizer: help to minimize the loss because it is a classificati

In [ ]:
model.fit(X_train, NN_y_train)

In [ ]:
#model.save('NN model.h5')
model = load_model('NN model.h5')

In [ ]:
predictions = model.predict(X_test)